In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Hiperparametreler
d_model = 512       # Gömme/Model boyutu
num_heads = 8       # Dikkat başı sayısı
num_layers = 6      # Kodlayıcı/Kod Çözücü katman sayısı
d_ff = 2048         # İleri Besleme Ağı (FFN) gizli boyutu
dropout_rate = 0.1
max_len = 1000      # Maksimum sıra uzunluğu

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # Denominatör hesaplaması (10000^(2i/d_model))
        den = torch.exp(-torch.arange(0, d_model, 2).float() * (math.log(10000.0) / d_model))

        # PE matrisi: [max_len, d_model]
        pos = torch.arange(0, max_len).float().unsqueeze(1) # [max_len, 1]

        # Açı hesaplaması
        pos_encoding = torch.zeros((max_len, d_model))

        # Çift indekslere sinüs, tek indekslere kosinüs uygula
        pos_encoding[:, 0::2] = torch.sin(pos * den)
        pos_encoding[:, 1::2] = torch.cos(pos * den)

        # [1, max_len, d_model] şekline getir ve kaydet
        self.pos_encoding = pos_encoding.unsqueeze(0)

    def forward(self, x):
        # x'in boyutları: [batch_size, seq_len, d_model]
        seq_len = x.size(1)
        # x + Konumsal Kodlama (gerekli uzunlukta kesilmiş)
        return x + self.pos_encoding[:, :seq_len, :].to(x.device)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=dropout_rate):
        super(MultiHeadAttention, self).__init__()

        self.attn = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        # PyTorch MultiheadAttention'da maskeleme biraz karmaşık olabilir.
        # Basitlik için burada ana arayüzü kullanıyoruz.

        # attn_output: [batch_size, seq_len, d_model]
        attn_output, _ = self.attn(query, key, value, attn_mask=mask, is_causal=False)

        # Kalıntı Bağlantısı ve Normalizasyon: norm(x + dropout(attn(x)))
        output = self.norm(query + self.dropout(attn_output))
        return output

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=dropout_rate):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads, dropout) # Öz-Dikkat

        # İleri Besleme Ağı (FFN)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # 1. Öz-Dikkat Alt Katmanı
        attn_output = self.mha(x, x, x, mask)
        # Kalıntı + Normalizasyon (PyTorch MHA içinde yapıldı, burada sadece FFN için yapalım)

        # 2. FFN Alt Katmanı
        ffn_output = self.ffn(attn_output)
        ffn_output = self.dropout2(ffn_output)

        # Kalıntı + Normalizasyon (ffn_output + attn_output)
        out = self.norm2(attn_output + ffn_output)

        return out

In [ ]:
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout_rate):
        super(SimpleTransformer, self).__init__()

        # Gömme Katmanı
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model)
        self.dropout = nn.Dropout(dropout_rate)

        # Kodlayıcı (Encoder)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout_rate)
            for _ in range(num_layers)
        ])

        # Çıkış katmanı (Örnek çıktı için)
        self.final_linear = nn.Linear(d_model, vocab_size)

    def forward(self, src_input, src_mask=None):
        # 1. Gömme ve Konumsal Kodlama
        x = self.embedding(src_input) # [batch_size, seq_len, d_model]
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # 2. Kodlayıcı Katmanları
        for layer in self.encoder_layers:
            x = layer(x, src_mask)

        # 3. Çıkış (basit bir sınıflandırma/tahmin katmanı)
        output = self.final_linear(x) # [batch_size, seq_len, vocab_size]
        return output

# --- Model Kullanımı ve Test ---

# Örnek sözlük boyutu (vocab_size)
vocab_size = 1000
batch_size = 64
seq_len = 50

# Giriş verisi (rastgele kelime indeksleri)
src_data = torch.randint(0, vocab_size, (batch_size, seq_len))

# Modeli oluştur
model = SimpleTransformer(vocab_size, d_model, num_layers, num_heads, d_ff, dropout_rate)

# Modeli çalıştır
output = model(src_data)

print(f"Giriş Verisi Şekli: {src_data.shape}")
print(f"Çıkış Tensör Şekli (Modelin Çıktısı): {output.shape}")
# [batch_size, seq_len, vocab_size] -> Her konumdaki kelime için tahmin skorları

Giriş Verisi Şekli: torch.Size([64, 50])
Çıkış Tensör Şekli (Modelin Çıktısı): torch.Size([64, 50, 1000])


In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=dropout_rate):
        super(DecoderLayer, self).__init__()

        # 1. Maskelenmiş Öz-Dikkat
        self.masked_mha = MultiHeadAttention(d_model, num_heads, dropout)

        # 2. Çapraz Dikkat (Kodlayıcı Çıktısına Dikkat)
        # Q=Decoder, K=V=Encoder
        self.cross_mha = MultiHeadAttention(d_model, num_heads, dropout)

        # 3. İleri Besleme Ağı (FFN)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

        # Normalizasyon ve Dropout katmanları
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, look_ahead_mask=None, padding_mask=None):

        # 1. Maskelenmiş Öz-Dikkat
        # x: [batch_size, tgt_seq_len, d_model]
        # look_ahead_mask: (Gelecek kelimeleri engeller)
        attn1 = self.masked_mha(x, x, x, mask=look_ahead_mask)
        out1 = self.norm1(x + self.dropout(attn1)) # Kalıntı + Normalizasyon

        # 2. Çapraz Dikkat
        # Q=out1, K=V=enc_output. enc_output'a padding_mask uygulanır.
        attn2 = self.cross_mha(out1, enc_output, enc_output, mask=padding_mask)
        out2 = self.norm2(out1 + self.dropout(attn2)) # Kalıntı + Normalizasyon

        # 3. FFN
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout(ffn_output)

        # Kalıntı + Normalizasyon
        out3 = self.norm3(out2 + ffn_output)

        return out3

In [ ]:
class FullTransformer(nn.Module):
    def __init__(self, input_vocab_size, target_vocab_size, d_model, num_layers, num_heads, d_ff, dropout_rate):
        super(FullTransformer, self).__init__()

        # Paylaşımlı bileşenler
        self.d_model = d_model

        # Kodlayıcı Tanımlaması
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.encoder_pos_encoding = PositionalEncoding(d_model)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout_rate)
            for _ in range(num_layers)
        ])

        # Kod Çözücü Tanımlaması
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.decoder_pos_encoding = PositionalEncoding(d_model)
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout_rate)
            for _ in range(num_layers)
        ])

        # Son Çıkış Katmanı
        self.final_linear = nn.Linear(d_model, target_vocab_size)

    def forward(self, src_input, tgt_input, src_mask=None, look_ahead_mask=None, padding_mask=None):

        # --- 1. Kodlayıcı (Encoder) Aşaması ---

        # Giriş gömme + Konumsal Kodlama
        enc_x = self.encoder_embedding(src_input) * math.sqrt(self.d_model)
        enc_x = self.encoder_pos_encoding(enc_x)

        # Kodlayıcı katmanlarından geçir
        encoder_output = enc_x
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output, src_mask)
            # encoder_output: Kaynak cümlenin bağlamsal temsili

        # --- 2. Kod Çözücü (Decoder) Aşaması ---

        # Hedef gömme + Konumsal Kodlama
        dec_x = self.decoder_embedding(tgt_input) * math.sqrt(self.d_model)
        dec_x = self.decoder_pos_encoding(dec_x)

        # Kod Çözücü katmanlarından geçir
        decoder_output = dec_x
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output,
                                   encoder_output, # Kodlayıcı çıktısı Çapraz Dikkat için
                                   look_ahead_mask,
                                   padding_mask)

        # --- 3. Çıkış Aşaması ---
        final_output = self.final_linear(decoder_output)

        return final_output

In [ ]:
# Kütüphaneleri yükleyelim
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import math

# Ayarlar
D_MODEL = 256        # Model boyutu
NHEAD = 4            # Dikkat başı sayısı
NUM_LAYERS = 3       # Encoder katman sayısı
DROPOUT = 0.1
MAX_LEN = 128        # Maksimum cümle uzunluğu
BATCH_SIZE = 32
VOCAB_MODEL = 'bert-base-uncased' # Tokenizer için temel BERT modeli

# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")

Kullanılan Cihaz: cpu


In [ ]:
print("\n--- 2. Veri Yükleme ve Tokenize Etme ---")

# 2.1. Veri Setini Yükle
# sst2: Basit İkili Duygu Analizi (0: Negatif, 1: Pozitif)
raw_datasets = load_dataset("glue", "sst2")

# 2.2. Tokenizer'ı Yükle
tokenizer = AutoTokenizer.from_pretrained(VOCAB_MODEL)
VOCAB_SIZE = tokenizer.vocab_size

# 2.3. Veri İşleme Fonksiyonu
def tokenize_function(examples):
    # 'sentence' alanını token'lara dönüştür
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

# 2.4. Tokenize Etme ve PyTorch Formatına Çevirme
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Sadece gerekli sütunları koru
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
# Sınıflandırma etiketi: 'label'ı 'labels' olarak yeniden adlandır (PyTorch'a uygun)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# PyTorch tensörlerine dönüştür
tokenized_datasets.set_format("torch")

# 2.5. DataLoader Oluşturma
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=BATCH_SIZE)

print(f"Sözlük Boyutu (VOCAB_SIZE): {VOCAB_SIZE}")
print(f"Eğitim Örneği Sayısı: {len(tokenized_datasets['train'])}")


--- 2. Veri Yükleme ve Tokenize Etme ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Sözlük Boyutu (VOCAB_SIZE): 30522
Eğitim Örneği Sayısı: 67349


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=DROPOUT, max_len=MAX_LEN):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)# [max_len-2, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0)) # [1, max_len, d_model]

    def forward(self, x):
        # x shape: [seq_len, batch_size, d_model]
        seq_len = x.size(0)
        batch_size = x.size(1)


        pe_to_add = self.pe[0, :seq_len, :]

        x = x + pe_to_add.unsqueeze(1).to(x.device)


        return self.dropout(x)

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, num_classes, dropout):
        super().__init__()

        # 4.1. Gömme ve Konumsal Kodlama
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        # 4.2. Ana Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4, # Genellikle d_model * 4 kullanılır
            dropout=dropout,
            batch_first=False # PyTorch'un varsayılanı
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 4.3. Sınıflandırma Katmanı
        # [CLS] token'ı (ilk token) ile temsil edilen cümle bağlamını kullanmak için tek bir D_MODEL vektörünü alacağız.
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, src, src_key_padding_mask=None):
        # src: [batch_size, seq_len] (Hugging Face'den gelen format)

        # 1. Gömme
        src_emb = self.token_embedding(src) * math.sqrt(D_MODEL) # [batch_size, seq_len, d_model]

        # Transpose src_emb to [seq_len, batch_size, d_model] for PositionalEncoder and TransformerEncoder
        src_emb = src_emb.transpose(0, 1)

        # 2. Konumsal Kodlama
        # src_emb: [seq_len, batch_size, d_model]
        src_emb = self.pos_encoder(src_emb)

        # 3. Transformer Encoder
        # src_key_padding_mask: [batch_size, seq_len] - PyTorch TransformerEncoder expects this shape for batch_first=False
        output = self.transformer_encoder(src_emb, src_key_padding_mask=src_key_padding_mask)

        # 4. Sınıflandırma
        # Sadece [CLS] token'ının çıktısını al (first token after transpose, which is output[0, :, :])
        cls_output = output[0, :, :]

        return self.classifier(cls_output)

In [ ]:
print("\n--- 5. Model Kurulumu ve Deneme Çalıştırması ---")

# Modelin Oluşturulması
num_classes = 2 # SST-2 için 2 sınıf (Pozitif/Negatif)
model = TransformerClassifier(
    VOCAB_SIZE, D_MODEL, NHEAD, NUM_LAYERS, num_classes, DROPOUT
).to(device)

# Kayıp fonksiyonu (Loss) ve Optimizasyon (Optimizer)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# --- Tek Bir Batch Üzerinde Deneme ---
model.train()
for batch in train_dataloader:
    # 1. Veri Hazırlığı
    input_ids = batch['input_ids'].to(device) # [batch_size, seq_len]
    labels = batch['labels'].to(device)       # [batch_size]

    # Padding Maskesi Oluşturma (0 indexi padding'dir)
    # src_key_padding_mask: [batch_size, seq_len]. 0 olan yerler True olmalı.
    padding_mask = (input_ids == tokenizer.pad_token_id)

    # 2. İleri Yayılım (Forward Pass)
    # output: [batch_size, num_classes]
    logits = model(input_ids, src_key_padding_mask=padding_mask)

    # 3. Kayıp Hesaplama
    loss = loss_fn(logits, labels)

    # 4. Geri Yayılım (Backward Pass)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Deneme için sadece ilk batch'i çalıştır ve dur
    print(f"Batch Giriş Şekli: {input_ids.shape}")
    print(f"Batch Çıktı (Logits) Şekli: {logits.shape}")
    print(f"İlk Batch Kayıp Değeri: {loss.item():.4f}")

    break

print("\n bir Transformer eğitim döngüsünün ilk adımı tamamlandı!")


--- 5. Model Kurulumu ve Deneme Çalıştırması ---


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Batch Giriş Şekli: torch.Size([32, 128])
Batch Çıktı (Logits) Şekli: torch.Size([32, 2])
İlk Batch Kayıp Değeri: 0.7801

Gerçek bir Transformer eğitim döngüsünün ilk adımı tamamlandı!


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm # İlerleme çubuğu için (opsiyonel ama faydalı)

# NOT: TransformerClassifier, PositionalEncoding, D_MODEL vb. daha önceki koddan tanımlanmış olmalıdır.

def evaluate_model(model, dataloader, device):
    """
    Verilen DataLoader üzerindeki model performansını değerlendirir ve sonuçları döndürür.
    """

    # Modeli değerlendirme moduna al (Dropout ve BatchNorm gibi katmanları dondurur)
    model.eval()

    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    # Gradyan hesaplamayı devre dışı bırak (Bellek ve hız kazancı için)
    with torch.no_grad():
        # tqdm ile ilerleme çubuğu ekleyelim
        for batch in tqdm(dataloader, desc="Model Değerlendirme"):

            # 1. Veri Hazırlığı
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            # Padding Maskesi Oluşturma (0 indexi padding'dir)
            # tokenizer'ın pad_token_id'si kullanılmalı
            padding_mask = (input_ids == tokenizer.pad_token_id)

            # 2. İleri Yayılım (Forward Pass)
            logits = model(input_ids, src_key_padding_mask=padding_mask)

            # 3. Kayıp Hesaplama (Doğrulama kaybını izlemek için)
            loss = nn.CrossEntropyLoss()(logits, labels)
            total_loss += loss.item() * input_ids.size(0) # Batch boyutunu hesaba kat

            # 4. Doğruluk (Accuracy) Hesaplama
            # Logit'lerdeki en büyük değere sahip indeksi bul (Tahmin edilen sınıf)
            _, predicted_classes = torch.max(logits, dim=1)

            # Doğru tahmin sayısını güncelle
            correct_predictions += (predicted_classes == labels).sum().item()
            total_samples += input_ids.size(0)

    # Sonuçları hesapla
    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    # Modeli tekrar eğitim moduna al
    model.train()

    return avg_loss, accuracy


# --- Model Değerlendirmeyi Başlatma ---

print("-" * 50)
print("Model Değerlendirme Başlatılıyor...")

# Önceki koddan tanımlanmış olmalıdır: model, eval_dataloader, device
# (Eğer önceki hücreleri çalıştırmadıysanız, model ve dataloader'ı yeniden tanımlamanız gerekir)

# Örnek kullanım (val_dataloader önceki kodda tanımlanmıştı):
validation_loss, validation_accuracy = evaluate_model(model, eval_dataloader, device)

print("-" * 50)
print(f"Doğrulama Kaybı (Validation Loss): {validation_loss:.4f}")
print(f"Doğrulama Doğruluğu (Validation Accuracy): {validation_accuracy:.4f} ({validation_accuracy*100:.2f}%)")
print("-" * 50)

--------------------------------------------------
Model Değerlendirme Başlatılıyor...


Model Değerlendirme: 100%|██████████| 28/28 [00:11<00:00,  2.51it/s]

--------------------------------------------------
Doğrulama Kaybı (Validation Loss): 0.7595
Doğrulama Doğruluğu (Validation Accuracy): 0.5092 (50.92%)
--------------------------------------------------


In [ ]:
import torch
import torch.optim as optim
import math

# --- 1. Öğrenme Oranı Zamanlayıcısı Sınıfı ---

class TransformerLRScheduler(optim.lr_scheduler._LRScheduler):
    """
    Orijinal Transformer makalesinde önerilen öğrenme oranı zamanlayıcısını uygular.
    """
    def __init__(self, optimizer, d_model, warmup_steps=4000, last_epoch=-1):
        # Hyperparametreler (Önceki kodunuzdan alınmalı)
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.num_step = 0 # Şu anki eğitim adımını takip eder
        super(TransformerLRScheduler, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        self.num_step += 1

        # Formülün ilk kısmı: Ters Karekök Azalımı (step_num^-0.5)
        decay_rate = self.num_step ** (-0.5)

        # Formülün ikinci kısmı: Warmup (step_num * warmup_steps^-1.5)
        warmup_rate = self.num_step * (self.warmup_steps ** (-1.5))

        # Ölçeklendirme faktörü (d_model^-0.5)
        scale = self.d_model ** (-0.5)

        # Nihai öğrenme oranı = scale * min(decay_rate, warmup_rate)
        new_lr = scale * min(decay_rate, warmup_rate)

        # Öğrenme oranını optimizer içindeki her parametre grubuna uygula
        return [new_lr for group in self.optimizer.param_groups]

# --- 2. Eğitim Hazırlığı (Güncellenmiş) ---

# Önceki kodunuzdan alınmalı: model, loss_fn, device, train_dataloader

# 1. Optimizasyon
# Not: Adam yerine AdamW tercih edilir (weight decay için)
optimizer = optim.AdamW(model.parameters(), lr=1.0, betas=(0.9, 0.98), eps=1e-9)
# lr=1.0 başlangıçta yüksek tutulur, zamanlayıcı gerçek değeri ayarlar.

# 2. Zamanlayıcı Tanımlaması
# d_model değeri önceki kodunuzdan alınmalı (örneğimizde 256)
scheduler = TransformerLRScheduler(optimizer, d_model=D_MODEL, warmup_steps=4000)

print("\n--- Güncellenmiş Eğitim Döngüsü (Zamanlayıcı Eklendi) ---")

# --- 3. Güncellenmiş Tek Batch Üzerinde Deneme ---
model.train()
max_batches = 5 # İlk 5 batch'i çalıştırıp LR değişimini gösterelim

for i, batch in enumerate(train_dataloader):
    if i >= max_batches:
        break

    # Veri Hazırlığı (Önceki koddan)
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    padding_mask = (input_ids == tokenizer.pad_token_id)

    # 1. İleri Yayılım (Forward Pass)
    logits = model(input_ids, src_key_padding_mask=padding_mask)
    loss = loss_fn(logits, labels)

    # 2. Geri Yayılım (Backward Pass)
    optimizer.zero_grad()
    loss.backward()

    # 3. Öğrenme Oranını Güncelleme (!!! KRİTİK DEĞİŞİKLİK !!!)
    optimizer.step()
    scheduler.step() # Zamanlayıcı, optimizer.step()'ten hemen sonra çağrılır.

    current_lr = optimizer.param_groups[0]['lr']

    print(f"Batch {i+1} | Adım: {scheduler.num_step} | LR: {current_lr:.6f} | Kayıp: {loss.item():.4f}")

print("\nÖğrenme oranı (LR) başarıyla eğitim adımlarına göre ayarlanmıştır.")


--- Güncellenmiş Eğitim Döngüsü (Zamanlayıcı Eklendi) ---
Batch 1 | Adım: 2 | LR: 0.000000 | Kayıp: 0.7408
Batch 2 | Adım: 3 | LR: 0.000001 | Kayıp: 0.7078
Batch 3 | Adım: 4 | LR: 0.000001 | Kayıp: 0.6938
Batch 4 | Adım: 5 | LR: 0.000001 | Kayıp: 0.8141
Batch 5 | Adım: 6 | LR: 0.000001 | Kayıp: 0.7986

Öğrenme oranı (LR) başarıyla eğitim adımlarına göre ayarlanmıştır.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler # Karma Hassasiyet için gerekli

# Önceki kodunuzdan tanımlanmış olmalıdır:
# model, loss_fn, optimizer, scheduler, train_dataloader, device, tokenizer

# --- 1. Gradyan Ölçekleyiciyi Tanımlama ---
# Bu nesne, FP16 gradyanlarının taşmasını (underflow/overflow) önler.
scaler = GradScaler()

print("\n--- Karma Hassasiyet ile Güncellenmiş Eğitim Döngüsü ---")

# --- 2. Güncellenmiş Eğitim Döngüsü ---
model.train()
max_batches = 5

for i, batch in enumerate(train_dataloader):
    if i >= max_batches:
        break

    # Veri Hazırlığı
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    padding_mask = (input_ids == tokenizer.pad_token_id)

    # 1. Sıfırlama
    optimizer.zero_grad()

    # !!! KRİTİK DEĞİŞİKLİK: autocast BLOĞU !!!
    # Bu blok içindeki işlemler FP16'da yapılır.
    with autocast():
        # İleri Yayılım (Forward Pass)
        logits = model(input_ids, src_key_padding_mask=padding_mask)
        loss = loss_fn(logits, labels)

    # 2. Geri Yayılım (Backward Pass)
    # Geri yayılımı scaler nesnesi üzerinden yap
    scaler.scale(loss).backward()

    # 3. Ağırlık Güncelleme (!!! KRİTİK DEĞİŞİKLİK !!!)
    # Gradyanların taşma yapıp yapmadığını kontrol eder.
    scaler.step(optimizer)

    # 4. Ölçekleyiciyi Güncelle ve LR'yi Ayarla
    scaler.update()
    scheduler.step()

    current_lr = optimizer.param_groups[0]['lr']

    print(f"Batch {i+1} | LR: {current_lr:.6f} | Kayıp (FP16): {loss.item():.4f}")

print("\nEğitim döngüsü artık Karma Hassasiyet kullanılarak hızlandırıldı.")

/tmp/ipython-input-3712326483.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/tmp/ipython-input-3712326483.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(



--- Karma Hassasiyet ile Güncellenmiş Eğitim Döngüsü ---
Batch 1 | LR: 0.000002 | Kayıp (FP16): 0.6558
Batch 2 | LR: 0.000002 | Kayıp (FP16): 0.7249
Batch 3 | LR: 0.000002 | Kayıp (FP16): 0.6381
Batch 4 | LR: 0.000002 | Kayıp (FP16): 0.6490
Batch 5 | LR: 0.000003 | Kayıp (FP16): 0.7350

Eğitim döngüsü artık Karma Hassasiyet kullanılarak hızlandırıldı.


In [ ]:
import torch
import torch.nn.functional as F

# --- Sabitler ---
BEAM_SIZE = 3      # Takip edilecek en iyi hipotez sayısı (Işın boyutu)
MAX_LEN_TGT = 50   # Maksimum çıktı cümle uzunluğu
PAD_IDX = 0        # Padding token indeksi
EOS_IDX = 2        # Cümle Sonu (End of Sentence) token indeksi

def beam_search_decode(model, src_input, src_mask, src_padding_mask, device):

    # Modeli değerlendirme moduna al
    model.eval()

    # 1. Kodlayıcı Çıktısını Hazırlama
    # Kaynak cümleyi bağlamsal temsile dönüştür (Memory Vektörü)
    memory = model.encode(src_input, src_mask, src_padding_mask) # [src_len, 1, d_model]

    # Bellek maskesini hazırla
    memory_key_padding_mask = src_padding_mask

    # 2. Başlangıç Hipotezlerini Oluşturma
    # Tüm hipotezler için Başlangıç token'ını (BOS) kullanıyoruz (Genellikle indeks 1)

    # Hipotez listesi: (kümülatif_log_olasılık, token_dizisi, tamamlandı_mı)
    # İlk hipotez: (0.0, [BOS_IDX])
    initial_hypothesis = (0.0, torch.LongTensor([1]).to(device), False)

    # Aktif hipotezler
    # Hipotezler, log olasılığa göre min-heap (küçükten büyüğe sıralı) olarak tutulur.
    # Ancak burada basitlik için liste olarak tutacağız.
    active_hypotheses = [initial_hypothesis]

    # Tamamlanmış hipotezler (EOS'a ulaşanlar)
    completed_hypotheses = []

    # --- 3. Adım Adım Üretim Döngüsü ---
    for i in range(MAX_LEN_TGT):
        if not active_hypotheses:
            break

        new_hypotheses = []

        # Her aktif hipotez için sonraki kelimeyi tahmin et
        for score, tokens, is_complete in active_hypotheses:
            if is_complete:
                new_hypotheses.append((score, tokens, True))
                continue

            # a. Kod Çözücü Girişini Hazırla
            # tokens: [1, mevcut_uzunluk] şeklinden [mevcut_uzunluk, 1] şekline getir (batch_first=False)
            tgt_input = tokens.unsqueeze(1)

            # b. Maske Oluştur: Gelecek token'ları engelle (Look-Ahead Mask)
            tgt_mask = model.transformer.generate_square_subsequent_mask(tgt_input.size(0)).to(device)

            # c. Kod Çözücü Çağrısı (Sadece son tokenın çıktısını al)
            # decode_output: [mevcut_uzunluk, 1, tgt_vocab_size]
            decode_output = model.decode(tgt_input, memory, tgt_mask, memory_key_padding_mask)

            # d. Son tokenın çıktısını al
            # log_probs: [tgt_vocab_size]
            log_probs = F.log_softmax(decode_output[-1, 0, :], dim=-1)

            # e. En İyi B Olasılığı Bul
            # Geçerli hipotezden sonraki en olası BEAM_SIZE kelimeyi al
            top_log_probs, top_indices = log_probs.topk(BEAM_SIZE)

            # f. Yeni Hipotezleri Oluştur
            for k in range(BEAM_SIZE):
                new_score = score + top_log_probs[k].item()
                new_token = top_indices[k].item()
                new_tokens = torch.cat([tokens, torch.LongTensor([new_token]).to(device)])

                # EOS kontrolü
                is_new_complete = (new_token == EOS_IDX)

                if is_new_complete:
                    completed_hypotheses.append((new_score / len(new_tokens), new_tokens, True))
                    # Not: Olasılık uzunluğa normalize edilir (Normalization by length)
                else:
                    new_hypotheses.append((new_score, new_tokens, False))

        # Hipotez listesini güncelle ve en iyi B tanesini koru
        # Tamamlanmamış hipotezleri kümülatif skora göre sırala ve en iyi BEAM_SIZE tanesini al
        active_hypotheses = sorted(new_hypotheses, key=lambda x: x[0], reverse=True)[:BEAM_SIZE]

    # 4. En İyi Sonucu Seçme
    # Eğer tamamlanmış hipotez yoksa, aktif olan en iyi hipotezi kullan.
    final_results = completed_hypotheses if completed_hypotheses else active_hypotheses

    # En yüksek normalize edilmiş skora sahip sonucu seç
    best_hypothesis = max(final_results, key=lambda x: x[0]) if final_results else None

    if best_hypothesis:
        # Sadece token dizisini döndür
        return best_hypothesis[1].tolist()
    else:
        return [EOS_IDX] # Çözüm bulunamadı